# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.pipeline.core import Pipeline

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-experiment'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [12]:
project_name = 'azureml-spaceship-titanic'
project_folder = os.getcwd().split(project_name)[0] + project_name

In [13]:
import sys
sys.path.append(project_folder)

from src.pipelines.preprocess import preprocess_data

df = pd.read_csv('../data/01_raw/train.csv')
processed_df = preprocess_data(df)

In [10]:
found = False
dataset_name = "Spaceship_Dataset"
description_text = "Data to predict which passengers are transported to an alternate dimension"


if dataset_name in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[dataset_name]

if not found:
        # Create AML Dataset and register it into Workspace
        dataset = Dataset.Tabular.register_pandas_dataframe(processed_df, ws.get_default_datastore(), dataset_name)   
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=dataset_name,
                                   description=description_text,
                                   create_new_version=True
                                )


df = dataset.to_pandas_dataframe()
df.describe()

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,People_in_Cabin_Num,People_in_Cabin_Deck,Family_Size,Group_Size
count,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000,8494.000000,8494.000000,8493.000000,8693.000000
mean,224.687617,458.077203,173.729169,311.138778,304.854791,8.037203,1952.105957,5.428117,2.035546
std,666.717663,1611.489240,604.696458,1136.705535,1145.717189,5.214669,963.741893,2.891901,1.596347
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,779.000000,3.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,2559.000000,5.000000,1.000000
75%,47.000000,76.000000,27.000000,59.000000,46.000000,12.000000,2794.000000,7.000000,3.000000
max,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000,28.000000,2794.000000,18.000000,8.000000


In [11]:
df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Transported', 'Cabin_Deck', 'Cabin_Side', 'Cabin_Region',
       'People_in_Cabin_Num', 'People_in_Cabin_Deck', 'Family_Size',
       'Group_Size', 'Age_Cat'],
      dtype='object')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [14]:
project_folder

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook250837/code/Users/odl_user_250837/azureml-spaceship-titanic'

In [15]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Transported",   
                             path = project_folder + '/automl-pipeline',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)


pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [16]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_2a1baf4c-ff77-4a66-b923-af7a09af430f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
